In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import datetime
import os.path
import backtrader as bt
import backtrader.indicators as btind
import pandas as pd
from pandas import Series, DataFrame
import math
import random


## Strategy

In [ ]:
# Create a Stratey
class TestStrategy(bt.Strategy):
    params = (
        ('maperiod', 15),
        ('printlog', False),
    )

    def log(self, txt, dt=None, doprint=False):
        """ Logging function fot this strategy"""
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close

        # To keep track of pending orders and buy price/commission
        self.order = None
        self.buyprice = None
        self.buycomm = None
        self.bar_executed = 0

        # Add a MovingAverageSimple indicator
        self.sma = bt.indicators.SimpleMovingAverage(self.datas[0], period=self.params.maperiod)

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price, order.executed.value,order.executed.comm)
                )

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            
            else:  # Sell
                self.log(
                    'SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price, order.executed.value, order.executed.comm)
                 )

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' % (trade.pnl, trade.pnlcomm))

    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

        # Check if we are in the market
        if not self.position:

            # Not yet ... we MIGHT BUY if ...
            if self.dataclose[0] > self.sma[0]:

                # BUY, BUY, BUY!!! (with all possible default parameters)
                self.log('BUY CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.buy()

        else:

            if self.dataclose[0] < self.sma[0]:
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log('SELL CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()

    def stop(self):
        self.log('(MA Period %2d) Ending Value %.2f' % (self.params.maperiod, self.broker.getvalue()), doprint=True)
        

# Simple Moving Average Strategy
A simple moving average crossover strategy; crossing of a fast and slow moving average generates buy/sell signals

In [ ]:
class SMAC(bt.Strategy):    
    params = {
        "fast": 20, 
        "slow": 50,                  
        "optim": False, 
        "optim_fs": (20, 50)
    }
 
    def __init__(self):
        """Initialize the strategy"""
        self.fastma = dict()
        self.slowma = dict()
        self.regime = dict()
 
        if self.params.optim:   
            self.params.fast, self.params.slow = self.params.optim_fs
 
        if self.params.fast > self.params.slow:
            raise ValueError(
                "A SMAC strategy cannot have the fast moving average's window be " + \
                 "greater than the slow moving average window.")
 
        for d in self.getdatanames():
            self.fastma[d] = bt.indicators.SimpleMovingAverage(
                self.getdatabyname(d), period=self.params.fast, plotname="FastMA: " + d
            )
            self.slowma[d] = bt.indicators.SimpleMovingAverage(
                self.getdatabyname(d), period=self.params.slow, plotname="SlowMA: " + d
            )
            self.regime[d] = self.fastma[d] - self.slowma[d]  
            
    def notify_trade(self, trade):
        if not trade.isclosed:
            return
        
 
    def next(self):
        """Define what will be done in a single step, including creating and closing trades"""
        for d in self.getdatanames():
            pos = self.getpositionbyname(d).size or 0
            if pos == 0:
                if self.regime[d][0] > 0 >= self.regime[d][-1]:    # Buy signal
                    self.buy(data=self.getdatabyname(d))
            else:    # We have an open position
                if self.regime[d][0] <= 0 < self.regime[d][-1]:    # Sell signal
                    self.sell(data=self.getdatabyname(d))

# Analyzer
A simple analyzer for account 

In [ ]:
class AcctStats(bt.Analyzer): 
    def __init__(self):
        self.start_val = self.strategy.broker.get_value()
        self.end_val = 0

    
    def start(self):
        self.ntrade = 0        
        
    def notify_trade(self, trade):
        self.ntrade += 1
 
    def stop(self):
        self.end_val = self.strategy.broker.get_value()
 
    def get_analysis(self):
        return {
            "start": self.start_val, 
            "end": self.end_val,
            "growth": self.end_val - self.start_val, 
            "return": self.end_val / self.start_val,
            "trades": self.ntrade
        }

# Test Strategy Test (In Sample)
Testing the original strategy with in-sample data to create a baseline.

In [ ]:
# Initialization & strategy
cerebro = bt.Cerebro()
cerebro.addstrategy(TestStrategy)

# Symbol
fname_symbol = 'CL'
folder_name = '5min'
suffix = '5min_20160103_20190405'

# Feed
df = pd.read_parquet(os.path.join(
        '../data/processed/{}/'.format(folder_name), 
        '{}_{}.parquet'.format(fname_symbol, suffix)
    ))
df = (df.resample('4h', label='left', base=18).agg({
        'Open': 'first', 
        'High': 'max', 
        'Low': 'min', 
        'Close': 'last', 
        'Volume': 'sum'
    }))
df.columns = [col_name.lower() for col_name in df.columns]
df = df.dropna()

# Set working datasets
data = math.floor(len(df) *.2)
InSampleData = bt.feeds.PandasData(dataname = df.iloc[0:(len(df) - data)])
OOSData = bt.feeds.PandasData(dataname = df.iloc[(len(df) - data):])

# Setup Cerebro
cerebro.adddata(InSampleData)

cerebro.broker.setcash(100000.0)
cerebro.addsizer(bt.sizers.FixedSize, stake=1000)
cerebro.broker.set_slippage_fixed(.01)

cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='mysharpe')
cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')

# Run output & Plotting
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
cerebro.run()
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

#cerebro.plot(volume=False , iplot=True)

# SMA Strategy Test (In Sample)
Using the same dataset as before (in-sample), run a baseline SMA strategy.

In [ ]:
# Initialization & strategy
cerebro = bt.Cerebro()
cerebro.addstrategy(SMAC)

# Setup Cerebro
cerebro.adddata(InSampleData)

cerebro.broker.setcash(100000.0)
cerebro.broker.set_slippage_fixed(.01)
cerebro.addsizer(bt.sizers.FixedSize, stake=1000)

# Print out the starting conditions
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

# Run over everything
cerebro.run()

# Print out the final result
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
cerebro.plot(volume=False , iplot=True)

# In-sample Optimization
Generate random combinations of fast and slow window lengths to test

In [ ]:
# Avoids duplication
windowset = set()

# Generate a bunch time windows for comparison.
# This will be used within the analyzer to get the 'best' strategy.
while len(windowset) < 40:
    f = random.randint(1, 10) * 5
    s = random.randint(1, 10) * 10
    if f > s:    # Cannot have the fast moving average have a longer window than the slow, so swap
        f, s = s, f
    elif f == s:    # Cannot be equal, so do nothing, discarding results
        pass
    windowset.add((f, s))
 
windows = list(windowset)
windows

# Create a cerebro entity
cerebro = bt.Cerebro(maxcpus=3)
strategies = cerebro.optstrategy(
    SMAC, 
    optim=True, 
    optim_fs=windows
)

# Setup Cerebro
cerebro.adddata(InSampleData)

cerebro.broker.setcash(100000.0)
cerebro.addsizer(bt.sizers.FixedSize, stake=1000)
cerebro.broker.set_slippage_fixed(.01)

#Analyzer
cerebro.addanalyzer(AcctStats)


%time res = cerebro.run()

# Store results of optimization in a DataFrame
return_opt = DataFrame({r[0].params.optim_fs: r[0].analyzers.acctstats.get_analysis() for r in res}
                      ).T.loc[:, ['start','end', 'growth', 'return', 'trades']]

# Sort my highest growth. This should be our best parameters to test against OOS.
ret = return_opt.sort_values("growth", ascending=False)
ret

# Optimized In Sample

In [ ]:
# using the best params from above, start plotting and displaying the info.
fast_opt, slow_opt = ret.iloc[0].name

optimized = bt.Cerebro()
optimized.adddata(InSampleData)
optimized.broker.setcash(100000.0)
optimized.addsizer(bt.sizers.FixedSize, stake=1000)
optimized.broker.set_slippage_fixed(.01)
optimized.addstrategy(SMAC, fast=fast_opt, slow=slow_opt)

optimized.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe')
optimized.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')
optimized.addanalyzer(AcctStats, _name='custom')

# Run over everything
res = optimized.run()
ISsharpe = res[0].analyzers.sharpe.get_analysis()
ISdrawdown = res[0].analyzers.drawdown.get_analysis()

# Print out the starting conditions
print('Starting Portfolio Value: %.2f' % optimized.broker.getvalue())
print('Final Portfolio Value: %.2f' % optimized.broker.getvalue())

In [ ]:
#optimized.plot(iplot=True, volume=False)

# Out of Sample 

In [ ]:
cerebro = bt.Cerebro()
cerebro.addstrategy(SMAC, fast=fast_opt, slow=slow_opt)
cerebro.adddata(OOSData)
cerebro.broker.setcash(100000.0)
cerebro.broker.set_slippage_fixed(.01)
cerebro.addsizer(bt.sizers.FixedSize, stake=1000)

cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe')
cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')
cerebro.addanalyzer(AcctStats, _name='custom')



# Run over everything
OOSresults = cerebro.run()


OOSsharpe = OOSresults[0].analyzers.sharpe.get_analysis()
OOSdrawdown = OOSresults[0].analyzers.drawdown.get_analysis()

print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

In [ ]:
#cerebro.plot(volume=True , iplot=True)

In [ ]:
print("In Sample Sharpe")
print(res[0].analyzers.sharpe.get_analysis())

print("Out of Sample Sharpe")
print(OOSresults[0].analyzers.sharpe.get_analysis())